#Import Packages

In [ ]:
!nvidia-smi

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torch.utils.data. sampler import SubsetRandomSampler, WeightedRandomSampler
from tqdm import tqdm
import copy
from typing import Counter
import os
import cv2
import shutil

In [ ]:
myseed = 2022
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

#Dowload Data

In [ ]:
!unzip -q './Test_set/test_0.zip'
!unzip -q './Test_set/test_1.zip'
!unzip -q './Test_set/test_2.zip'
!unzip -q './Test_set/test_3.zip'
!unzip -q './Test_set/test_4.zip'
!unzip -q './Test_set/test_5.zip'
!unzip -q './Test_set/test_6.zip'
!unzip -q './Test_set/test_7.zip'
!unzip -q './Test_set/test_8.zip'
!unzip -q './Test_set/test_9.zip'
!unzip -q './Test_set/test_a.zip'
!unzip -q './Test_set/test_b.zip'
!unzip -q './Test_set/test_c.zip'
!unzip -q './Test_set/test_d.zip'
!unzip -q './Test_set/test_e.zip'
!unzip -q './Test_set/test_f.zip'

In [ ]:
os.mkdir('testset')

In [ ]:
datas = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f']

In [ ]:
for data in datas:
    for i in range(len(os.listdir('./test_'+data))):
        img = os.listdir('./test_'+data)[0]
        shutil.move('./test_'+data+'/'+img, './testset')

#Dataset

In [ ]:
class Dataset():
    
    def __init__(self, images_dir,):
        self.ids = os.listdir(images_dir)
        self.images = [os.path.join(images_dir, image_id) for image_id in self.ids]
    
    def __getitem__(self, i):
        
        image = cv2.imread(self.images[i])
        id = self.ids[i]

        test_transforms = transforms. Compose([transforms.ToPILImage(),
                                               transforms.Resize((300, 300)),
                                               transforms. ToTensor(),
                                               transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                   std=[0.229, 0.224, 0.225]),])
        image = test_transforms(image)
            
        return image, id
        
    def __len__(self):
        return len(self.ids)

In [ ]:
test_set = Dataset('./testset')

#DataLoadet

In [ ]:
test_loader = DataLoader(test_set, batch_size=64 ,shuffle=False, num_workers=2)

#Load weight

In [ ]:
save_path = os.path.join('./AI_CUP_Agricultural', 'model_weight_b3.pth')

model_2 = models.efficientnet_b3()
model_2.classifier._modules['1'] = nn.Linear(1536, 14)

## load weight
state = torch.load(save_path)
model_2.load_state_dict(state['state_dict'])
model_2.to(device)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
save_path = os.path.join('./AI_CUP_Agricultural', 'model_weight_b4.pth')

model_1 = models.efficientnet_b4()
model_1.classifier._modules['1'] = nn.Linear(1792, 14)

## load weight
state = torch.load(save_path)
model_1.load_state_dict(state['state_dict'])
model_1.to(device)

#Predict

In [ ]:
id = []
predicts = []


model_1.eval()
model_2.eval()

with torch.no_grad():
    for i, (image,ids) in enumerate(tqdm(test_loader)):
        image = image.to(device)
        y_hat_1 = model_1(image)
        y_hat_2 = model_2(image)
        y_hat = y_hat_1*0.5 + y_hat_2*0.5
        _, pred = torch.max(y_hat.data, 1)
        for y in pred.cpu().detach().numpy():
            predicts.append(y)
        for i in ids:
            id.append(i)

In [ ]:
import csv

classes = ['banana', 'bareland', 'carrot', 'corn', 'dragonfruit', 'garlic', 'guava', 'peanut', 'pineapple', 'pumpkin', 'rice', 'soybean', 'sugarcane', 'tomato']

with open('output.csv', 'w') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['image_filename','label'])
  for i in range(len(test_set)):
      writer.writerow([str(id[i]),str(classes[predicts[i]])])